# **Task - 03 Advanced Level**
# **Develop a Neural Network that can read Handwriting**
---

## Importing the libraries

In [1]:
!pip install idx2numpy

In [2]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

## Importing the dataset

### Importing set of images

In [3]:
trainig_set_images = 'train-images.idx3-ubyte'
X_train_images = idx2numpy.convert_from_file(trainig_set_images)
# print(X_train_images[0])

In [4]:
X_train_images.shape

(60000, 28, 28)

In [5]:
test_set_images = 't10k-images.idx3-ubyte'
X_test_images = idx2numpy.convert_from_file(test_set_images)
# print(X_test_images[0])

In [6]:
X_test_images.shape

(10000, 28, 28)

### Importing set of labels

In [7]:
def read_idx1_file(file):
    with open(file, 'rb') as f:
        magic_number = int.from_bytes(f.read(4), byteorder='big')
        num_items = int.from_bytes(f.read(4), byteorder='big')
        labels = [int.from_bytes(f.read(1), byteorder='big') for _ in range(num_items)]
    return labels

In [8]:
training_set_labels = 'train-labels.idx1-ubyte'
y_train_labels = read_idx1_file(training_set_labels)
print("y_train labels: ", y_train_labels)

y_train labels:  [5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1, 1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0, 4, 5, 6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9, 0, 4, 6, 7, 4, 6, 8, 0, 7, 8, 3, 1, 5, 7, 1, 7, 1, 1, 6, 3, 0, 2, 9, 3, 1, 1, 0, 4, 9, 2, 0, 0, 2, 0, 2, 7, 1, 8, 6, 4, 1, 6, 3, 4, 5, 9, 1, 3, 3, 8, 5, 4, 7, 7, 4, 2, 8, 5, 8, 6, 7, 3, 4, 6, 1, 9, 9, 6, 0, 3, 7, 2, 8, 2, 9, 4, 4, 6, 4, 9, 7, 0, 9, 2, 9, 5, 1, 5, 9, 1, 2, 3, 2, 3, 5, 9, 1, 7, 6, 2, 8, 2, 2, 5, 0, 7, 4, 9, 7, 8, 3, 2, 1, 1, 8, 3, 6, 1, 0, 3, 1, 0, 0, 1, 7, 2, 7, 3, 0, 4, 6, 5, 2, 6, 4, 7, 1, 8, 9, 9, 3, 0, 7, 1, 0, 2, 0, 3, 5, 4, 6, 5, 8, 6, 3, 7, 5, 8, 0, 9, 1, 0, 3, 1, 2, 2, 3, 3, 6, 4, 7, 5, 0, 6, 2, 7, 9, 8, 5, 9, 2, 1, 1, 4, 4, 5, 6, 4, 1, 2, 5, 3, 9, 3, 9, 0, 5, 9, 6, 5, 7, 4, 1, 3, 4, 0, 4, 8, 0, 4, 3, 6, 8, 7, 6, 0, 9, 7, 5, 7, 2, 1, 1, 6, 8, 9, 4, 1, 5, 2, 2, 9, 0, 3, 9, 6, 7, 2, 0, 3

In [9]:
test_set_labels = 't10k-labels.idx1-ubyte'
y_test_labels = read_idx1_file(test_set_labels)
print("y_test labels: ", y_test_labels)

y_test labels:  [7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9, 8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7, 9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4, 7, 1, 2, 4, 0, 2, 7, 4, 3, 3, 0, 0, 3, 1, 9, 6, 5, 2, 5, 9, 2, 9, 3, 0, 4, 2, 0,

## Preprocessing the data

In [10]:
X_train_images = X_train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test_images = X_test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train_labels = to_categorical(y_train_labels, num_classes=10)
y_test_labels = to_categorical(y_test_labels, num_classes=10)

## Building the Convolutional Neural Network (CNN)

### Initialising the CNN

In [11]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [12]:
cnn.add(layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [28, 28, 1]))

### Step 2 - Pooling

In [13]:
cnn.add(layers.MaxPool2D(pool_size = 2, strides = 2))

### Adding hidden layer 1 and 2

In [14]:
cnn.add(layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
cnn.add(layers.MaxPool2D(pool_size = 2, strides = 2))
cnn.add(layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))

### Step 3 - Flattening

In [15]:
cnn.add(layers.Flatten())

### Step 4 - Full Connection

In [16]:
cnn.add(layers.Dense(units = 64, activation = 'relu')) # hidden layer 3
cnn.add(layers.Dense(units = 10, activation = 'softmax')) # output layer

## Training the Convolutional Neural Network (CNN)

### Compiling the CNN

In [17]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [18]:
cnn.fit(X_train_images, y_train_labels, batch_size = 64, epochs = 10, validation_data=(X_test_images, y_test_labels))

Epoch 1/10
938/938 [==============================] - 73s 76ms/step - loss: 0.1828 - accuracy: 0.9439 - val_loss: 0.0581 - val_accuracy: 0.9814
Epoch 2/10
938/938 [==============================] - 78s 84ms/step - loss: 0.0505 - accuracy: 0.9845 - val_loss: 0.0352 - val_accuracy: 0.9883
Epoch 3/10
938/938 [==============================] - 69s 74ms/step - loss: 0.0357 - accuracy: 0.9891 - val_loss: 0.0334 - val_accuracy: 0.9884
Epoch 4/10
938/938 [==============================] - 63s 67ms/step - loss: 0.0303 - accuracy: 0.9904 - val_loss: 0.0370 - val_accuracy: 0.9892
Epoch 5/10
938/938 [==============================] - 63s 68ms/step - loss: 0.0233 - accuracy: 0.9928 - val_loss: 0.0321 - val_accuracy: 0.9898
Epoch 6/10
938/938 [==============================] - 62s 67ms/step - loss: 0.0194 - accuracy: 0.9939 - val_loss: 0.0294 - val_accuracy: 0.9904
Epoch 7/10
938/938 [==============================] - 61s 65ms/step - loss: 0.0159 - accuracy: 0.9950 - val_loss: 0.0300 - val_accuracy:

## Evaluating the Convolutional Neural Network (CNN)

In [24]:
test_loss, test_acc = cnn.evaluate(X_test_images, y_test_labels)
print("Test loss: {:.4f}".format(test_loss))
print("Test accuracy: {:.2f} %".format(test_acc*100))

313/313 [==============================] - 5s 16ms/step - loss: 0.0351 - accuracy: 0.9905
Test loss: 0.0351
Test accuracy: 99.05 %
